# 9. Final System

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight
import math
import random
import pickle
from IPython.display import display
RANDOM_STATE = 123
np.random.seed(123)

c:\Users\danih\anaconda3\envs\MasterIA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Final System

## Load data

In [2]:
df_gym = pd.read_csv('data/modified_gym_members_exercise_tracking.csv')
df_heart_rates = pd.read_csv('data/gym_members_heart_rates.csv')
df_users = pd.read_csv('data/User Listening History_modified.csv')
df_music = pd.read_csv('data/Million Song Dataset kaggle/Music Info.csv')

In [3]:
df_music_info = df_music[['track_id', 'name', 'artist', 'energy', 'duration_ms']]

In [4]:
id_to_cluster = pd.read_csv('data/track_clusters.csv', index_col=0).iloc[:, 0]

In [5]:
df_gym.shape

(247, 15)

## Stage 1 - Energy Calculator

In [6]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [7]:
class FuzzyController:
    def __init__(self):
        self.bpm_antecedent = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'Normalized BPM')
        self.bpm_variation_antecedent = ctrl.Antecedent(np.arange(-0.2, 0.21, 0.01), 'Normalized BPM Variation')
        self.energy_consequent = ctrl.Consequent(np.arange(-0.17, 1.18, 0.01), 'Energy')
        #self.energy_consequent = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'Energy')

        self.bpm_antecedent['Very Light'] = fuzz.trapmf(self.bpm_antecedent.universe, [0.00, 0.00, 0.54, 0.60])
        self.bpm_antecedent['Light'] = fuzz.trapmf(self.bpm_antecedent.universe, [0.54, 0.60, 0.61, 0.67])
        self.bpm_antecedent['Moderate'] = fuzz.trapmf(self.bpm_antecedent.universe, [0.61, 0.67, 0.70, 0.84])
        self.bpm_antecedent['Vigorous'] = fuzz.trapmf(self.bpm_antecedent.universe, [0.70, 0.84, 0.93, 0.99])
        self.bpm_antecedent['Near Maximal'] = fuzz.trapmf(self.bpm_antecedent.universe, [0.93, 0.99, 1.00, 1.00])

        self.bpm_variation_antecedent['Negative'] = fuzz.trapmf(self.bpm_variation_antecedent.universe, [-0.2, -0.2, -0.15, -0.05])
        self.bpm_variation_antecedent['Zero'] = fuzz.trapmf(self.bpm_variation_antecedent.universe, [-0.15, -0.05, 0.05, 0.15])
        self.bpm_variation_antecedent['Positive'] = fuzz.trapmf(self.bpm_variation_antecedent.universe, [0.05, 0.15, 0.2, 0.21])

        
        self.energy_consequent['Low'] = fuzz.trapmf(self.energy_consequent.universe, [-0.2, -0.2, 0.125, 0.375])
        self.energy_consequent['Medium'] = fuzz.trapmf(self.energy_consequent.universe, [0.125, 0.5, 0.5, 0.875])
        self.energy_consequent['High'] = fuzz.trapmf(self.energy_consequent.universe, [0.625, 0.875, 1.21, 1.21])


        self.energy_consequent.defuzzify_method = 'centroid'

        # Rules:
        # Rule Intensity_zone Variation ⇒ Energy
        # R1 Very_Light – ⇒ High
        # R1 Light Negative ⇒ High
        # R1 Light Zero ⇒ High
        # R1 Moderate Negative ⇒ High

        # R2 Light Positive ⇒ Medium
        # R2 Moderate Zero ⇒ Medium
        # R2 Moderate Positive ⇒ Medium
        # R2 Vigorous Negative ⇒ Medium
        # R2 Vigorous Zero ⇒ Medium

        # R3 Near_maximal – ⇒ Low
        # R3 Vigorous Positive ⇒ Low

        rule1 = ctrl.Rule(antecedent= (self.bpm_antecedent['Very Light'] |
                        (self.bpm_antecedent['Light'] & self.bpm_variation_antecedent['Negative']) |
                        (self.bpm_antecedent['Light'] & self.bpm_variation_antecedent['Zero']) |
                        (self.bpm_antecedent['Moderate'] & self.bpm_variation_antecedent['Negative'])),
                        consequent=self.energy_consequent['High'])
        
        rule2 = ctrl.Rule(antecedent=((self.bpm_antecedent['Light'] & self.bpm_variation_antecedent['Positive']) |
                                (self.bpm_antecedent['Moderate'] & self.bpm_variation_antecedent['Zero']) |
                                (self.bpm_antecedent['Moderate'] & self.bpm_variation_antecedent['Positive']) |
                                (self.bpm_antecedent['Vigorous'] & self.bpm_variation_antecedent['Negative'])) |
                                (self.bpm_antecedent['Vigorous'] & self.bpm_variation_antecedent['Zero']),
                                consequent=self.energy_consequent['Medium'])
        
        rule3 = ctrl.Rule(antecedent=(self.bpm_antecedent['Near Maximal'] |
                        (self.bpm_antecedent['Vigorous'] & self.bpm_variation_antecedent['Positive'])),
                        consequent=self.energy_consequent['Low'])

        energy_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
        self.energy_sim = ctrl.ControlSystemSimulation(energy_ctrl)

    def calculate_energy(self, bpm, bpm_variation, age, plot_consequent=False, plot_antecedent=False):
        hr_max = 208 - 0.7 * age # Paper: Age-Predicted Maximal Heart Rate Revisited

        bpm_normalized = bpm  / hr_max
        bpm_variation_normalized = bpm_variation / hr_max

        self.energy_sim.input['Normalized BPM'] = bpm_normalized
        self.energy_sim.input['Normalized BPM Variation'] = bpm_variation_normalized
        self.energy_sim.compute()

        
        if plot_consequent:
            self.energy_consequent.view(sim=self.energy_sim)
        if plot_antecedent:
            self.bpm_antecedent.view(sim=self.energy_sim)
            self.bpm_variation_antecedent.view(sim=self.energy_sim)
        
        return self.energy_sim.output['Energy']

    def view_bpm_antecedent(self):
        self.bpm_antecedent.view()
    
    def view_bpm_variation_antecedent(self):
        self.bpm_variation_antecedent.view()

    def view_energy_consequent(self):
        self.energy_consequent.view()


In [8]:
class EnergyCalculator:
    def __init__(self, df_gym_member, df_heart_rates, fuzzy_controller=None):
        self.user_age = df_gym_member['Age']
        self.df_heart_rates = df_heart_rates
        self.sesion_minute = 0
        if fuzzy_controller is None:
            self.fuzzy_controller = FuzzyController()
        else:
            self.fuzzy_controller = fuzzy_controller

    def calculate_energy(self, plot_consequent=False, plot_antecedent=False):
        if self.sesion_minute == 0:
            return 0.6 # Default energy for the first song
        if self.sesion_minute >= len(self.df_heart_rates):
            return -1 # Indicates that the session has ended
        bpm_current = self.df_heart_rates[self.sesion_minute]
        bpm_variation = bpm_current - self.df_heart_rates[self.sesion_minute - 1]
        print(f"Calculating energy for session minute {self.sesion_minute}")
        print(f"Previous BPM: {self.df_heart_rates[self.sesion_minute - 1]}, Current BPM: {bpm_current}, BPM Variation: {bpm_variation}")
        return self.fuzzy_controller.calculate_energy(self.df_heart_rates[self.sesion_minute], bpm_variation, self.user_age, plot_consequent, plot_antecedent)
    
    def pass_song_duration(self, song_duration=2): # Song duration in minutes
        self.sesion_minute += song_duration
        if self.sesion_minute >= len(self.df_heart_rates):
            return -1
        return self.sesion_minute
    
    def get_session_minute(self):
        return self.sesion_minute

    def view_bpm_antecedent(self):
        self.fuzzy_controller.view_bpm_antecedent()
    
    def view_bpm_variation_antecedent(self):
        self.fuzzy_controller.view_bpm_variation_antecedent()
    
    def view_energy_consequent(self):
        self.fuzzy_controller.view_energy_consequent()

## Stage 2 - Recommendation System

Interaction matrix

In [9]:
df_users_agg = df_users.groupby('user_id')['playcount'].agg(
    #total_playcount='sum',
    max_playcount='max'
).reset_index()
df_users_agg = df_users_agg.rename(columns={'playcount': 'max_playcount'})

df_users_rating = df_users.merge(df_users_agg, on='user_id')

user_codes, user_uniques = pd.factorize(df_users['user_id'])
track_codes, track_uniques = pd.factorize(df_users['track_id'])

In [10]:
interaction_matrix_user_item_original = csr_matrix(
    (df_users_rating['playcount'], (user_codes, track_codes)),
    shape=(len(user_uniques), len(track_uniques))
)

interaction_matrix_user_item = bm25_weight(interaction_matrix_user_item_original, K1=1.2, B=0.75).tocsr()

### Matrix Factorization: Alternating Least Squares (ALS)

In [11]:
from implicit.als import AlternatingLeastSquares

In [12]:
class ALSRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, als_model=None):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info

        if als_model is None:
            self.als_model = AlternatingLeastSquares(factors=100, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
            self.als_model.fit(self.interaction_matrix)
        else:
            self.als_model = als_model

        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index

        user_items = self.interaction_matrix.tocsr()[user_index]


        top_n_recommendations_indexes, top_n_recommendations_scores = self.als_model.recommend(user_index, user_items, N=n, filter_already_liked_items=True)

        # for i in range(len(top_n_recommendations_indexes)):
        #     print(f"Track ID: {self.track_uniques[top_n_recommendations_indexes[i]]}, Similarity: {top_n_recommendations_scores[i]}")


        track_ids = self.track_uniques[top_n_recommendations_indexes].tolist()
        
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        self.recommendations = [(track_id, energy, similarity, False) for (track_id, energy), similarity in zip(df_filtered.itertuples(index=False, name=None), top_n_recommendations_scores)]
        return self.recommendations

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

### Kmeans content-based filtering

In [13]:
class KmeansContentBasedRecommender:
    def __init__(self, id_to_cluster):
        self.id_to_cluster = id_to_cluster
        self.recommendations = None
    
    def make_cluster_recommendation(self, user_history):
        clusters = self.id_to_cluster[user_history]
        cluster_counts = clusters.value_counts()
        self.recommended_cluster = cluster_counts / len(clusters)
        return self.recommended_cluster

    def get_recommended_cluster(self):
        if self.recommended_cluster is None:
            raise ValueError("No cluster recommendation available. Please call make_cluster_recommendation first.")
        return self.recommendations
    

### Hybrid recommender

In [14]:
class HybridRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, df_users, id_to_cluster, als_recommender = None, content_based_recommender = None, alpha = 2):
        if als_recommender is not None:
            self.collaborative_als_recommender = als_recommender
        else:
            self.collaborative_als_recommender = ALSRecommender(interaction_matrix, track_uniques, df_music_info)
        
        if content_based_recommender is not None:
            self.content_based_recommender = content_based_recommender  
        else:
            self.content_based_recommender = KmeansContentBasedRecommender(id_to_cluster)

        self.df_music_info = df_music_info
        self.df_users = df_users
        self.id_to_cluster = id_to_cluster
        self.alpha = alpha  # Alpha is a parameter to control the influence of content-based recommendations
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    
    def make_recommendations(self, user_index, n=100):

        user_id = self.df_users['user_id'].unique()[user_index]
        user_history = self.df_users[self.df_users['user_id'] == user_id]['track_id']
        collaborative_recomendations = self.collaborative_als_recommender.make_recommendations(user_index, n)
        content_based_cluster_recommendation = self.content_based_recommender.make_cluster_recommendation(user_history)
        self.recommendations = []
        
        #We will apply a penalization to the collaborative filtering recommendation based on the user cluster preferences obtained by the content-based recommendation
        for track_id, energy, similarity, has_been_recommended in collaborative_recomendations:
            cluster_presence = 0 #Default multiplier. Used if the song's cluster is not in the user's cluster preferences (content-based recommendation)
            song_cluster = self.id_to_cluster[track_id]
            if song_cluster in content_based_cluster_recommendation.index:
                cluster_presence = content_based_cluster_recommendation[song_cluster]
            
            #print(track_id, song_cluster, multiplier)

            self.recommendations.append((track_id, energy, similarity + cluster_presence * self.alpha, has_been_recommended)) # confidence = colab_conficence + cluster_presence * self.alpha
        self.recommendations = sorted(self.recommendations, key=lambda x: x[2], reverse=True)  # Sort new similarity


    def make_recommendations_only_collaborative(self, user_index, n=100):
        self.recommendations = self.collaborative_als_recommender.make_recommendations(user_index, n)
    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")
    
    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations
    
    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

## Final System

In [15]:
class MusicRecommender2Stages:
    def __init__(self, energy_calculator, hybrid_recommender, user_index, df_music_info):
        self.energy_calculator = energy_calculator
        self.hybrid_recommender = hybrid_recommender
        self.user_index = user_index
        self.df_music_info = df_music_info


    def make_recommendations(self, n=100):
        self.hybrid_recommender.make_recommendations(self.user_index, n)
        
    
    def recommend_song(self):
        current_minute = self.energy_calculator.get_session_minute()
        energy = self.energy_calculator.calculate_energy()
        if energy == -1:
            return current_minute, None # Session has ended
        print(f"Energy level needed for recommendation: {energy}")
        song_id, _ = self.hybrid_recommender.recommend_song(energy)
        song_duration_minutes = self.df_music_info[self.df_music_info['track_id'] == song_id]['duration_ms'].values[0] // 60000
        self.energy_calculator.pass_song_duration(song_duration_minutes)
        return current_minute, self.df_music_info[self.df_music_info['track_id'] == song_id]
    
    def pass_song_duration(self, song_duration=2):
        return self.energy_calculator.pass_song_duration(song_duration)
    
    def get_recommendations(self):
        return self.hybrid_recommender.get_recommendations()
    
    def get_recommendations_ids(self):
        return self.hybrid_recommender.get_recommendations_ids()
    
    def get_recommendations_info(self):
        return self.hybrid_recommender.get_recommendations_info()
    

## Example

In [16]:
df_gym.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
1,36,Male,70.3,1.72,174,169,73,1.49,1385.0,Cardio,21.3,2.3,3,2,23.76
2,40,Female,69.7,1.51,189,141,64,1.27,895.0,Cardio,30.6,1.9,3,2,30.57
3,28,Male,101.8,1.84,169,136,64,1.08,808.0,Cardio,29.7,2.7,3,1,30.07
4,57,Male,112.5,1.61,195,165,61,1.24,1013.0,Cardio,22.1,2.7,3,2,43.40


In [17]:
client_used = 2
client_heart_rates = df_heart_rates[df_heart_rates['User_ID'] == client_used]['Heart_Rate'].tolist()

In [18]:
energy_calculator = EnergyCalculator(df_gym.iloc[client_used], client_heart_rates)

In [19]:
with open('models/als_model.pkl', 'rb') as f:
    als_model = pickle.load(f)

In [20]:
als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model)

In [21]:
hybrid_recommender = HybridRecommender(interaction_matrix_user_item, track_uniques, df_music_info, df_users, id_to_cluster, als_recommender=als_recommender)

In [22]:
music_recommender_2_stages = MusicRecommender2Stages(energy_calculator, hybrid_recommender, client_used, df_music_info)
music_recommender_2_stages.make_recommendations(n=100)
music_recommender_2_stages.get_recommendations_info().head(20)

,track_id,name,artist,energy,duration_ms
0,TRJOLVL128F4262D6E,Comfort Eagle,Cake,0.782,220226
1,TRZPETU128F427704B,Short Skirt/Long Jacket,Cake,0.897,204160
2,TRDPXKD128F42634BD,Shadow Stabbing,Cake,0.572,187266
3,TRJEBWL128F428A2A1,"Never, Never Gonna Give You Up",Barry White,0.806,478626
4,TRMWSBM128F428D305,Stickshifts And Safetybelts,Cake,0.682,128653
5,TRWFJOB128F4298317,Newjack,Justice,0.856,216880
6,TRWMGXO128F428A28F,"Mahna, Mahna",Cake,0.483,174066
7,TRRIHLA128F424819D,Rock 'n' Roll Lifestyle,Cake,0.455,254066
8,TRMPSSY128F428D398,Where Would I Be?,Gwen Stefani,0.769,198280
9,TRAFWEV128F428D391,Alpha Beta Parking Lot,Cake,0.651,210066


In [23]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Energy level needed for recommendation: 0.6
Session minute: 0
Recommended song:


,track_id,name,artist,energy,duration_ms
14161,TRDPXKD128F42634BD,Shadow Stabbing,Cake,0.572,187266


In [24]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 3
Previous BPM: 126, Current BPM: 119, BPM Variation: -7
Energy level needed for recommendation: 0.5585871045349716
Recommended song:


,track_id,name,artist,energy,duration_ms
9480,TRUNJWL128F42481A4,Mr. Mastodon Farm,Cake,0.555,327600


In [25]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 8
Previous BPM: 116, Current BPM: 144, BPM Variation: 28
Energy level needed for recommendation: 0.21549483982758996
Recommended song:


,track_id,name,artist,energy,duration_ms
9157,TRFETPH128F4280AE3,End of the Movie,Cake,0.255,109826


In [26]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 9
Previous BPM: 144, Current BPM: 105, BPM Variation: -39
Energy level needed for recommendation: 0.9396037910018212
Recommended song:


,track_id,name,artist,energy,duration_ms
14034,TRZPETU128F427704B,Short Skirt/Long Jacket,Cake,0.897,204160


In [27]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 12
Previous BPM: 125, Current BPM: 168, BPM Variation: 43
Energy level needed for recommendation: 0.048953295211557016
Recommended song:


,track_id,name,artist,energy,duration_ms
29455,TREAQSX128E07818CA,One More Time (Romanthony's Unplugged),Daft Punk,0.0967,220586


In [28]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 15
Previous BPM: 162, Current BPM: 143, BPM Variation: -19
Energy level needed for recommendation: 0.6362366449906278
Recommended song:


,track_id,name,artist,energy,duration_ms
14121,TRMWSBM128F428D305,Stickshifts And Safetybelts,Cake,0.682,128653


In [29]:
minute, df_recommended_song = music_recommender_2_stages.recommend_song()
#print(f"Session minute: {minute}")
if df_recommended_song is None:
    print("Session has already ended.")
else:
    print(f"Recommended song:")
    display(df_recommended_song)

Calculating energy for session minute 17
Previous BPM: 168, Current BPM: 126, BPM Variation: -42
Energy level needed for recommendation: 0.9537698412698409
Recommended song:


,track_id,name,artist,energy,duration_ms
2987,TRFJIOA128E0781CB7,Body Movin',Beastie Boys,0.92,331360
